## Trénovanie SpaCy modelov
Autor: Juraj Dedič
Spúšťané v prostredí kaggle.com

In [2]:
#Import all required libraries
import spacy
import random
import time
import numpy as np
import sys
from spacy import displacy
from itertools import chain
import matplotlib.pyplot as plt 
from matplotlib.ticker import MaxNLocator
import json

In [ ]:
!pip install spacy-transformers

Create base config file from here:https://spacy.io/usage/training#quickstart

In [4]:
!python -m spacy init fill-config /kaggle/input/config/base_config_spancat.cfg config_spancat.cfg

✔ Auto-filled config with all values
✔ Saved config
config_spancat.cfg
You can now add your data and train your pipeline:
python -m spacy train config_spancat.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Train using CLI

In [9]:
!python -m spacy train /kaggle/input/config/config.cfg --verbose --gpu-id 0 --output ./ner/training/ --paths.train /kaggle/input/callsigns-more-classes-v5/train.spacy --paths.dev /kaggle/input/callsigns-more-classes-v5/validation.spacy

[2023-05-05 17:24:39,699] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
✔ Created output directory: ner/training
ℹ Saving to output directory: ner/training
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-05-05 17:24:44,224] [INFO] Set up nlp object from config
[2023-05-05 17:24:44,236] [DEBUG] Loading corpus from path: /kaggle/input/callsigns-more-classes-v5/validation.spacy
[2023-05-05 17:24:44,237] [DEBUG] Loading corpus from path: /kaggle/input/callsigns-more-classes-v5/train.spacy
[2023-05-05 17:24:44,237] [INFO] Pipeline: ['transformer', 'ner']
[2023-05-05 17:24:44,243] [INFO] Created vocabulary
[2023-05-05 17:24:44,244] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS ex

## Train SpanCat

In [12]:
!python -m spacy train /kaggle/input/config/config_spancat_20ep.cfg --verbose --gpu-id 0 --output ./spancat/training/ --paths.train /kaggle/input/callsigngs-v4/train_spancat.spacy --paths.dev /kaggle/input/callsigngs-v4/validation_spancat.spacy

[2023-03-24 18:29:35,700] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
ℹ Saving to output directory: spancat/training
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-03-24 18:29:36,111] [INFO] Set up nlp object from config
[2023-03-24 18:29:36,123] [DEBUG] Loading corpus from path: /kaggle/input/callsigngs-v4/validation_spancat.spacy
[2023-03-24 18:29:36,124] [DEBUG] Loading corpus from path: /kaggle/input/callsigngs-v4/train_spancat.spacy
[2023-03-24 18:29:36,124] [INFO] Pipeline: ['transformer', 'spancat']
[2023-03-24 18:29:36,130] [INFO] Created vocabulary
[2023-03-24 18:29:36,131] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaMod

Load our model and test data

In [17]:
!cat /kaggle/working/spancat/training/model-best/meta.json

{
  "lang":"en",
  "name":"pipeline",
  "version":"0.0.0",
  "spacy_version":">=3.5.1,<3.6.0",
  "description":"",
  "author":"",
  "email":"",
  "url":"",
  "license":"",
  "spacy_git_version":"Unknown",
  "vectors":{
    "width":0,
    "vectors":0,
    "keys":0,
    "name":null,
    "mode":"default"
  },
  "labels":{
    "transformer":[

    ],
    "spancat":[
      "CALLSIGN"
    ]
  },
  "pipeline":[
    "transformer",
    "spancat"
  ],
  "components":[
    "transformer",
    "spancat"
  ],
  "disabled":[

  ],
  "performance":{
    "spans_sc_f":0.8998035363,
    "spans_sc_p":0.9308943089,
    "spans_sc_r":0.8707224335,
    "transformer_loss":140.8263847161,
    "spancat_loss":1223.8323513996
  }
}

In [ ]:
#import docbin
import sys
print(sys.executable)
import spacy
from spacy.tokens import DocBin
from spacy import displacy
from spacy.scorer import Scorer
from spacy.training import Example

ner = spacy.load(R"/kaggle/working/demo/training/model-best") #load the best model

db_test_correct = DocBin().from_disk(R"/kaggle/input/callsigns-more-classes-v5/test.spacy") #load the validation data

docs_test_correct = db_test_correct.get_docs(ner.vocab) #get the validation data
# test_sentences = [x[0] for x in TEST_DATA[0:100]] # extract the sentences from [sentence, entity]

scorer = Scorer()
examples = []

for doc_correct in docs_test_correct:
    text = doc_correct.text
    doc_ner = ner(text) # predict the entities

    #create the example
    example = Example(doc_ner, doc_correct)
    examples.append(example)


    # # print the predicted entities
    # displacy.render(doc_ner, style="ent", jupyter=True)
    # # print the correct entities
    # displacy.render(doc_correct,jupyter=True, style = "ent")

res = scorer.score(examples)
print(res)

Test with a sentence

In [10]:
import shutil
shutil.make_archive("output", 'zip', "./ner_demo/training/model-last/")

'/kaggle/working/output.zip'

In [14]:
!python --version

Python 3.7.12


In [ ]:
ner = spacy.load(R"ner/training/model-best") #load the best model
icao_callsigns_file = open("callsigns-icao-edit.json")
icao_callsigns = json.load(icao_callsigns_file)

doc = ner("""Singapore Seven Two Nine Eight contact departures good night""")
displacy.render(doc,jupyter=True, style = "ent")
print(doc.ents.count)

In [ ]:
ner = spacy.load(R"/kaggle/working/demo/training/model-best") #load the best model

doc = ner("""Singapore Seven Two Nine Eight contact departures good night""")
displacy.render(doc,jupyter=True, style = "ent")
print(doc.ents.count)